In [1]:
import ebooklib
from dissection_table.database.sources_formatting import *
from dissection_table.operations.sources import *
import time
from ebooklib import epub
from dissection_table.database.engine import init_db,get_db_session
from dissection_table.database.models import ParagraphSimilarity

from constants import *
from dissection_table.database.ask_db import *
from dissection_table.database.db_interface import DBInterface
from dissection_table.operations.version_corpus import Corpus
await init_db(CONN_STRING)
import time
DBInterface.initialize_engine_and_session(CONN_STRING)

2025-08-06 17:13:25.214116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754522005.230057  457043 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754522005.234653  457043 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754522005.247550  457043 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754522005.247571  457043 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754522005.247572  457043 computation_placer.cc:177] computation placer alr

Database 'dissection_table' already exists.
Ensuring database tables exist...
Database tables checked/created.
Database initialization complete.
DBInterface: SQLAlchemy engine and AsyncSessionLocal initialized.


In [2]:
await create_similarity_data()

In [3]:
# async for session in get_db_session():
#     try:
#         # --- Step 3: Call open_request with the session ---
#         # Pass the 'session' object as the first argument, then your SQL query.
#         print("Attempting to drop tables...")
#         await open_request(session, "DROP TABLE version CASCADE;")
#         await open_request(session, "DROP TABLE paragraph CASCADE;")
#         await open_request(session, "DROP TABLE paragraph_similarity CASCADE;")
        
#         print("Tables dropped successfully.")
#     except Exception as e:
#         print(f"An error occurred while dropping tables: {e}")
#     finally:
#         # The 'break' statement ensures we exit the loop after using the session once.
#         break

Attempting to drop tables...
Tables dropped successfully.


In [2]:
s = time.time()
await feed_database()
e = time.time()
print(e-s)

ollama_start_embeddings
ollama_end_embeddings
TIME ELAPSED FOR ALL EMBEDDINGS: 3.9464618007342023
umap_start_reducer


/home/jon/anaconda3/envs/nlp/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


umap_end_reducer
TIME ELAPSED FOR ALL EMBEDDINGS: 0.3780235409736633
ALL PARAGRAPHS EMBEDDING START
Generating embeddings for raw texts...
TIME ELAPSED FOR ALL PARAGRAPH_EMB: 0.017762521902720135
Applying UMAP reduction to raw text embeddings...


/home/jon/anaconda3/envs/nlp/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/home/jon/anaconda3/envs/nlp/lib/python3.12/site-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


TIME ELAPSED FOR ALL UMAP: 0.05569907029469808
Assigning text embeddings and UMAP to version data...
Successfully performed bulk insert for 9 items in Version.
Successfully performed bulk insert for 12895 items in Paragraph.
280.3135869503021


In [3]:
async with DBInterface(Version).get_session() as session:
    result = await open_request(session, """select * from paragraph_similarity
                                                where source_version_name = 'spanish_1'
                                                and target_version_name = 'english_1' limit(6);""", fetch_as_dict = True)
    print(f"The count is: {result}")

The count is: [{'id': 34735, 'source_version_name': 'spanish_1', 'source_n_paragraph': 0, 'target_version_name': 'english_1', 'target_n_paragraph': 0, 'rank': 0}, {'id': 34736, 'source_version_name': 'spanish_1', 'source_n_paragraph': 0, 'target_version_name': 'english_1', 'target_n_paragraph': 5, 'rank': 1}, {'id': 34737, 'source_version_name': 'spanish_1', 'source_n_paragraph': 0, 'target_version_name': 'english_1', 'target_n_paragraph': 977, 'rank': 2}, {'id': 34759, 'source_version_name': 'spanish_1', 'source_n_paragraph': 1, 'target_version_name': 'english_1', 'target_n_paragraph': 1, 'rank': 0}, {'id': 34760, 'source_version_name': 'spanish_1', 'source_n_paragraph': 1, 'target_version_name': 'english_1', 'target_n_paragraph': 664, 'rank': 1}, {'id': 34761, 'source_version_name': 'spanish_1', 'source_n_paragraph': 1, 'target_version_name': 'english_1', 'target_n_paragraph': 1134, 'rank': 2}]


In [8]:
'ópopo'

'ópopo'

In [9]:
sources.keys()

dict_keys(['portuguese_1', 'spanish_1', 'spanish_2', 'english_1', 'english_2', 'italian_1', 'french_1', 'german_1', 'turkish_1'])

In [10]:
len(sources.keys())

9

In [ ]:
await open_request_("select count(*) from paragraph") 

In [3]:
{'popo':52}.values()

dict_values([52])

In [4]:
a = sources["turkish_1"]

In [6]:
a = list(a.get_items_of_type(ebooklib.ITEM_DOCUMENT))

In [28]:
whole_text = BeautifulSoup(a[2].get_body_content(), 'html.parser')

In [29]:
paragraphs = [para.get_text() for para in whole_text.find_all('p')][:-2]

In [33]:
paragraphs[-1]

'Damiana’ya yaslanarak yürümeye çalıştı. Bir iki adım attıktan sonra yıkıldı, içi yanıyordu; ama tek kelime çıkmıyordu ağzından. Gövdesi usulca yere çarptı. Pedro Paramo bir taş yığını gibi dağılıverdi.'

In [11]:
async def create_corpus_instance(version_name: str):
    """
    Helper function to get a database session and create a Corpus instance.
    """
    
    async for session in get_db_session():
        try:
            print(f"Attempting to create Corpus instance for version: {version_name}")
            corpus_instance = await Corpus.create(session, version_name)
            print(f"Successfully created Corpus instance for {version_name}.")
            return corpus_instance
        except ValueError as e:
            print(f"Error creating Corpus instance: {e}")
            return None
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return None

In [12]:
async for session in get_db_session():
    portuguese_1 = await Corpus.create(session, 'portuguese_1')
    spanish_1    = await Corpus.create(session, 'spanish_1')
    spanish_2    = await Corpus.create(session, 'spanish_2')
    english_1    = await Corpus.create(session, 'english_1')
    english_2    = await Corpus.create(session, 'english_2')
    italian_1    = await Corpus.create(session, 'italian_1')    
    french_1     = await Corpus.create(session, 'french_1')    
    german_1     = await Corpus.create(session, 'german_1')
    turkish_1     = await Corpus.create(session, 'turkish_1')  
    
    

In [13]:
print(portuguese_1.author)
print(spanish_1   .author)
print(spanish_2   .author)
print(english_1   .author)
print(english_2   .author)
print(italian_1   .author)
print(french_1    .author)
print(german_1    .author)
print(turkish_1    .author)

Eric Nepomuceno
Juan Rulfo
Juan Rulfo
Margaret Sayers Peden
Douglas J. Weatherford
Paolo Collo
Gabriel Iaculli
Dagmar Ploetz
Tomris Uyar


In [14]:
print(portuguese_1.year)
print(spanish_1   .year)
print(spanish_2   .year)
print(english_1   .year)
print(english_2   .year)
print(italian_1   .year)
print(french_1    .year)
print(german_1    .year)
print(turkish_1    .year)


2009
1955
2011
1994
2023
2014
2005
2008
1983


In [15]:
print(portuguese_1.editorial)
print(spanish_1   .editorial)
print(spanish_2   .editorial)
print(english_1   .editorial)
print(english_2   .editorial)
print(italian_1   .editorial)
print(french_1    .editorial)
print(german_1    .editorial)
print(turkish_1    .editorial)


Edições BestBolso
www.elejandria.com
feather
Grove Press
Grove Press
Giulio Einaudi editore s.p.a.
Éditions Gallimard
Carl Hanser Verlag Munich
Can Yayinlari


In [16]:
print(portuguese_1.ISBN)
print(spanish_1   .ISBN)
print(spanish_2   .ISBN)
print(english_1   .ISBN)
print(english_2   .ISBN)
print(italian_1   .ISBN)
print(french_1    .ISBN)
print(german_1    .ISBN)
print(turkish_1    .ISBN)


9788577991167
None
None
9780802133908
9780802160935
9788858440247
9782070379538
9783446230668
None


In [17]:
print(portuguese_1.metadata[:50])
print(spanish_1   .metadata[:50])
print(spanish_2   .metadata[:50])
print(english_1   .metadata[:50])
print(english_2   .metadata[:50])
print(italian_1   .metadata[:50])
print(french_1    .metadata[:50])
print(german_1    .metadata[:50])
print(turkish_1    .metadata[:50])

#print(english_2.metadata[:50])

########## ITEM 2
Pedro Páramo
Juan Rulfo (1917-19
NO_METADATA
########## ITEM 2


Pedro Páramo se publicó en 195
########## ITEM 0
Juan Rulfo

########## ITEM 2
‘A strange, brooding novel … Gre
########## ITEM 2
Juan Rulfo
Traduzione di Paolo C
########## ITEM 2
JUAN RULFO

PEDRO PÁRAMO

roman

### META_ONE 
Juan RULFO
Pedro
Páramo
Roman
08 * 2
NO_METADATA


In [18]:
print(portuguese_1.text[:50])
print(spanish_1   .text[:50])
print(spanish_2   .text[:50])
print(english_1   .text[:50])
print(english_2   .text[:50])
print(italian_1   .text[:50])
print(french_1    .text[:50])
print(german_1    .text[:50])
print(turkish_1    .text[:50])


VIM A COMALA porque me disseram que aqui vivia meu

Vine a Comala porque me dijeron que acá vivía mi 
Vine a Comala porque me dijeron que acá vivía mi p
I came to Comala because I had been told that my f
I CAME TO COMALA because I was told my father live
Venni a Comala perché mi avevano detto che mio pad
Je suis venu à Comala parce que j’ai appris que mo
Ich bin nach Comala gekommen, weil mir gesagt wurd
Comala’ya babamı aramaya geldim; dediklerine bakıl


In [19]:
print(portuguese_1.n_words)
print(spanish_1   .n_words)
print(spanish_2   .n_words)
print(english_1   .n_words)
print(english_2   .n_words)
print(italian_1   .n_words)
print(french_1    .n_words)
print(german_1    .n_words)
print(german_1    .n_words)


34842
33375
33369
35692
36959
33682
37236
35014
35014


In [20]:
print(portuguese_1.n_paragraphs)
print(spanish_1   .n_paragraphs)
print(spanish_2   .n_paragraphs)
print(english_1   .n_paragraphs)
print(english_2   .n_paragraphs)
print(italian_1   .n_paragraphs)
print(french_1    .n_paragraphs)
print(german_1    .n_paragraphs)
print(turkish_1    .n_paragraphs)


1447
1444
1447
1464
1447
1444
1388
1413
1401


In [21]:
print(await portuguese_1.n_paragraph(6))
print('####')
print(await spanish_1.n_paragraph(5))
print('####')
print(await spanish_2 .n_paragraph(5))
print('###')
print(await english_1.n_paragraph(6))
print('####')
print(await english_2.n_paragraph(5))
print('####')

print(await italian_1.n_paragraph(5))
print('####')

print(await french_1.n_paragraph(5))
print('####')

print(await german_1.n_paragraph(5))
print('######')
print(await turkish_1    .n_paragraph(4))

aquele era o tempo da canícula quando o ar de agosto sopra quente envenenado pelo odor apodrecido das flores do sabãodemacaco
####
era ese tiempo de la canícula cuando el aire de agosto sopla caliente envenenado por el olor podrido de las saponarias
####
era ese tiempo de la canícula cuando el aire de agosto sopla caliente envenenado por el olor podrido de las saponarias
###
it was during the dog days the season when the august wind blows hot venomous with the rotten stench of saponaria blossoms
####
it was during the dog days of summer when the august winds blow hot tainted by the rotting smell of saponaria flowers
####
quello era il tempo della canicola quando l’aria d’agosto soffia bollente avvelenata dall’odore putrido delle saponarie
####
c’était pendant les jours caniculaires où souffle le vent d’août brûlant corrompu par l’odeur putride des savonniers
####
es war die zeit der hundstage wenn der augustwind heiß bläst vergiftet vom fauligen geruch der seifenkrautblüten
######
yazı

In [22]:
print(await portuguese_1.n_paragraph_umap(7))
print(await spanish_1   .n_paragraph_umap(6))
print(await spanish_2   .n_paragraph_umap(6))
print(await english_1   .n_paragraph_umap(7))
print(await english_2   .n_paragraph_umap(6))
print(await italian_1   .n_paragraph_umap(6))
print(await french_1    .n_paragraph_umap(6))
print(await german_1    .n_paragraph_umap(6))
print(await turkish_1    .n_paragraph_umap(4))

#printawait (await english_2.n_paragraph_umap(5))

[8.888029, 13.999101, 1.7514042]
[8.889835, 13.998077, 1.7508804]
[8.887817, 13.998568, 1.7533444]
[8.862125, 13.998639, 1.7619712]
[8.868191, 14.008827, 1.7657896]
[8.890052, 14.005718, 1.7528273]
[8.902617, 13.985499, 1.7415849]
[8.900331, 13.981123, 1.7424586]
[8.302185, 11.33894, 0.046419103]


aquele era o tempo da canícula quando o ar de agosto sopra quente envenenado pelo odor apodrecido das flores do sabãodemacaco
####
era ese tiempo de la canícula cuando el aire de agosto sopla caliente envenenado por el olor podrido de las saponarias
####
era ese tiempo de la canícula cuando el aire de agosto sopla caliente envenenado por el olor podrido de las saponarias
###
it was during the dog days the season when the august wind blows hot venomous with the rotten stench of saponaria blossoms
####
it was during the dog days of summer when the august winds blow hot tainted by the rotting smell of saponaria flowers
####
quello era il tempo della canicola quando l’aria d’agosto soffia bollente avvelenata dall’odore putrido delle saponarie
####
c’était pendant les jours caniculaires où souffle le vent d’août brûlant corrompu par l’odeur putride des savonniers
####
es war die zeit der hundstage wenn der augustwind heiß bläst vergiftet vom fauligen geruch der seifenkrautblüten


In [23]:
await portuguese_1.all_umap()

array([[ 8.88839   , 12.631456  ,  1.2556543 ],
       [ 8.925372  , 12.635689  ,  1.0434043 ],
       [11.370134  ,  7.0011287 , -1.8914076 ],
       ...,
       [11.484121  ,  7.5869236 ,  2.8810632 ],
       [11.952439  ,  5.41203   ,  0.5405135 ],
       [ 6.897244  , 11.93779   ,  0.18802793]], dtype=float32)

In [24]:
await spanish_1.all_umap()

array([[ 8.88839   , 12.631456  ,  1.2556543 ],
       [ 8.925372  , 12.635689  ,  1.0434043 ],
       [11.370134  ,  7.0011287 , -1.8914076 ],
       ...,
       [11.484121  ,  7.5869236 ,  2.8810632 ],
       [11.952439  ,  5.41203   ,  0.5405135 ],
       [ 6.897244  , 11.93779   ,  0.18802793]], dtype=float32)

In [26]:
await spanish_2.all_umap()

array([[ 8.860416  , 12.633042  ,  1.149422  ],
       [11.377602  ,  6.991102  , -1.8875843 ],
       [11.512602  , 10.873918  ,  0.71966803],
       ...,
       [11.460613  ,  7.613543  ,  2.8808527 ],
       [11.8757925 ,  5.4972334 ,  0.4168108 ],
       [ 6.873864  , 11.939403  ,  0.1736829 ]], dtype=float32)

In [27]:
await english_1.all_umap()

array([[ 8.770067  , 12.637219  ,  1.2290548 ],
       [11.373321  ,  6.9945846 , -1.8766241 ],
       [11.546927  , 10.80217   ,  0.7385166 ],
       ...,
       [11.415986  ,  7.6135826 ,  2.865452  ],
       [11.826331  ,  5.325422  ,  0.40770677],
       [ 6.862834  , 11.924385  ,  0.18789004]], dtype=float32)

In [28]:
import numpy as np
from scipy.spatial.distance import cdist
from typing import Dict, List, Tuple

In [29]:
matrices_dict = {
    'portuguese_1':    await portuguese_1.all_umap(),
    'spanish_1'   :    await spanish_1 .all_umap(),
    'spanish_2'   :    await spanish_2  .all_umap(),
    'english_1'   :    await english_1.all_umap(),
    'english_2'   :    await english_2 .all_umap(),
    'italian_1'   :    await italian_1 .all_umap(),
    'french_1'    :    await french_1 .all_umap(),
    'german_1'    :    await german_1 .all_umap(),
    'turkish_1'   :    await turkish_1 .all_umap(),
}

In [39]:
def find_most_similar_rows(matrices: Dict[str, np.ndarray]) -> Dict[str, Dict[str, List[int]]]:

    results: Dict[str, Dict[str, List[int]]] = {}
    
    for source_name, source_matrix in matrices.items():
        for i, source_row in enumerate(source_matrix):
            row_key = f'{source_name}#{i}'
            results[row_key] = {}
            
            for target_name, target_matrix in matrices.items():
                if source_name == target_name:
                    continue
                distances = cdist(source_row.reshape(1, -1), target_matrix, 'cosine')[0]
                similarities = 1 - distances
                num_target_rows = target_matrix.shape[0]
                num_to_get = min(3, num_target_rows)
                top_3_indices = np.argsort(similarities)[-num_to_get:][::-1]
                
                results[row_key][target_name] = top_3_indices.tolist()

    return results

In [31]:
a = find_most_similar_rows(matrices_dict)

In [38]:
a['spanish_1#3']

{'portuguese_1': [4, 1396, 1004],
 'spanish_2': [3, 1396, 647],
 'english_1': [4, 1413, 1311],
 'english_2': [3, 1396, 647],
 'italian_1': [3, 1393, 961],
 'french_1': [3, 1336, 604],
 'german_1': [3, 1364, 642],
 'turkish_1': [1350, 1324, 1244]}

In [ ]:
a.keys()

In [ ]:
await portuguese_1.all_umap()


In [ ]:
await corpus_instance.n_paragraph_umap(5)


In [ ]:
corpus_instance.n_paragraphs

In [ ]:
corpus_instance.metadata[:50]


In [27]:
corpus_instance.version

'german_1'

In [28]:
corpus_instance.author

'Dagmar Ploetz'

In [29]:
corpus_instance.year

2008

In [30]:
corpus_instance.editorial

'Carl Hanser Verlag Munich'

In [33]:
corpus_instance.ISBN

9783446230668

In [53]:
corpus_instance.metadata[:50]

'### META_ONE \nJuan RULFO\nPedro\nPáramo\nRoman\n08 * 2'

In [35]:
corpus_instance.text[:100]

'Ich bin nach Comala gekommen, weil mir gesagt wurde, dass hier mein Vater lebt, ein gewisser Pedro P'

In [36]:
corpus_instance.n_words

35014

In [37]:
corpus_instance.n_paragraphs

1413

In [38]:
await corpus_instance.word_freq()

OrderedDict([('und', 977),
             ('ich', 947),
             ('sie', 908),
             ('die', 871),
             ('er', 679),
             ('das', 671),
             ('der', 586),
             ('nicht', 481),
             ('es', 468),
             ('zu', 426),
             ('du', 422),
             ('den', 361),
             ('auf', 331),
             ('ist', 326),
             ('in', 319),
             ('sich', 315),
             ('war', 285),
             ('dass', 279),
             ('ein', 278),
             ('mir', 245),
             ('wie', 231),
             ('von', 228),
             ('als', 226),
             ('mit', 212),
             ('was', 202),
             ('so', 200),
             ('aber', 198),
             ('mich', 192),
             ('an', 188),
             ('hat', 181),
             ('dem', 177),
             ('da', 176),
             ('dann', 169),
             ('ihr', 153),
             ('noch', 153),
             ('wir', 151),
             ('ihm', 150),
 

In [39]:
await corpus_instance.int_to_word()

{0: 'und',
 1: 'ich',
 2: 'sie',
 3: 'die',
 4: 'er',
 5: 'das',
 6: 'der',
 7: 'nicht',
 8: 'es',
 9: 'zu',
 10: 'du',
 11: 'den',
 12: 'auf',
 13: 'ist',
 14: 'in',
 15: 'sich',
 16: 'war',
 17: 'dass',
 18: 'ein',
 19: 'mir',
 20: 'wie',
 21: 'von',
 22: 'als',
 23: 'mit',
 24: 'was',
 25: 'so',
 26: 'aber',
 27: 'mich',
 28: 'an',
 29: 'hat',
 30: 'dem',
 31: 'da',
 32: 'dann',
 33: 'ihr',
 34: 'noch',
 35: 'wir',
 36: 'ihm',
 37: 'ihn',
 38: 'schon',
 39: 'habe',
 40: 'wenn',
 41: 'um',
 42: 'hatte',
 43: 'nur',
 44: 'sagte',
 45: 'pedro',
 46: 'aus',
 47: 'eine',
 48: 'dich',
 49: 'man',
 50: 'doch',
 51: 'hier',
 52: 'im',
 53: 'ja',
 54: 'sein',
 55: 'haben',
 56: 'dir',
 57: 'für',
 58: 'uns',
 59: 'immer',
 60: 'auch',
 61: 'nach',
 62: 'jetzt',
 63: 'ihnen',
 64: 'wieder',
 65: 'páramo',
 66: 'dort',
 67: 'ihre',
 68: 'bin',
 69: 'vor',
 70: 'nichts',
 71: 'mehr',
 72: 'meine',
 73: 'wird',
 74: 'bei',
 75: 'einen',
 76: 'don',
 77: 'hast',
 78: 'sind',
 79: 'sagen',
 80: 'n

In [40]:
await corpus_instance.word_to_int()

{'und': 0,
 'ich': 1,
 'sie': 2,
 'die': 3,
 'er': 4,
 'das': 5,
 'der': 6,
 'nicht': 7,
 'es': 8,
 'zu': 9,
 'du': 10,
 'den': 11,
 'auf': 12,
 'ist': 13,
 'in': 14,
 'sich': 15,
 'war': 16,
 'dass': 17,
 'ein': 18,
 'mir': 19,
 'wie': 20,
 'von': 21,
 'als': 22,
 'mit': 23,
 'was': 24,
 'so': 25,
 'aber': 26,
 'mich': 27,
 'an': 28,
 'hat': 29,
 'dem': 30,
 'da': 31,
 'dann': 32,
 'ihr': 33,
 'noch': 34,
 'wir': 35,
 'ihm': 36,
 'ihn': 37,
 'schon': 38,
 'habe': 39,
 'wenn': 40,
 'um': 41,
 'hatte': 42,
 'nur': 43,
 'sagte': 44,
 'pedro': 45,
 'aus': 46,
 'eine': 47,
 'dich': 48,
 'man': 49,
 'doch': 50,
 'hier': 51,
 'im': 52,
 'ja': 53,
 'sein': 54,
 'haben': 55,
 'dir': 56,
 'für': 57,
 'uns': 58,
 'immer': 59,
 'auch': 60,
 'nach': 61,
 'jetzt': 62,
 'ihnen': 63,
 'wieder': 64,
 'páramo': 65,
 'dort': 66,
 'ihre': 67,
 'bin': 68,
 'vor': 69,
 'nichts': 70,
 'mehr': 71,
 'meine': 72,
 'wird': 73,
 'bei': 74,
 'einen': 75,
 'don': 76,
 'hast': 77,
 'sind': 78,
 'sagen': 79,
 'nacht

In [41]:
await corpus_instance.all_paragraphs()

{0: 'ich bin nach comala gekommen weil mir gesagt wurde dass hier mein vater lebt ein gewisser pedro páramo meine mutter hat mir das gesagt und ich habe ihr versprochen ihn gleich nach ihrem tod aufzusuchen ich habe ihr die hände gedrückt um das zu bekräftigen denn sie lag im sterben und ich hätte alles versprochen versäume nicht ihn zu besuchen trug sie mir auf er heißt so und so ich bin mir sicher dass es ihn freuen wird dich kennenzulernen',
 1: 'und da konnte ich nicht anders ich sagte ja das würde ich tun und ich sagte es so oft dass ich es auch dann noch sagte als ich meine hände nur mit mühe aus ihren toten händen befreien konnte',
 2: 'davor hatte sie noch gesagt bettle ihn ja nicht an fordere was uns zusteht das was er mir schuldig war und mir nie gegeben hat lass ihn teuer bezahlen dass er uns im stich gelassen hat mein sohn',
 3: 'das werde ich tun mutter',
 4: 'aber ich dachte nicht daran mein versprechen zu halten bis ich auf einmal voller träume war und die illusionen mit

In [42]:
await corpus_instance.all_embeddings()

array([[-0.01783206,  0.05276284, -0.03501383, ...,  0.06045781,
         0.02529268, -0.00240648],
       [-0.01628715, -0.02913761, -0.0347627 , ...,  0.02775041,
        -0.01645291,  0.00058036],
       [-0.01188705,  0.02024717, -0.064407  , ...,  0.02749525,
        -0.01820893,  0.00886313],
       ...,
       [-0.05071397,  0.01639335, -0.04944724, ...,  0.07861377,
        -0.02434452,  0.04517431],
       [ 0.00974161,  0.06658779, -0.01725288, ...,  0.0379482 ,
         0.01078561, -0.01099419],
       [ 0.0175803 ,  0.0263428 , -0.03796491, ...,  0.01481179,
         0.01454384, -0.00092772]], dtype=float32)

In [43]:
await corpus_instance.all_umap()

array([[ 8.806368 ,  3.6587615,  9.38037  ],
       [10.446485 ,  4.946808 ,  7.8122945],
       [10.530425 ,  1.9185033,  5.1525707],
       ...,
       [ 6.108304 ,  3.4906046,  7.8199406],
       [10.226859 ,  3.9487581,  0.8549479],
       [10.528792 ,  1.4552535, 12.544558 ]], dtype=float32)

In [44]:
await corpus_instance.n_paragraph(5)

'es war die zeit der hundstage wenn der augustwind heiß bläst vergiftet vom fauligen geruch der seifenkrautblüten'

In [45]:
await corpus_instance.n_paragraph_embedding(5)

[-0.042746283,
 0.02667222,
 -0.038828056,
 -0.008486553,
 -0.043108694,
 -0.01857907,
 -0.013023041,
 0.017583594,
 0.049355816,
 0.01371225,
 -0.010261172,
 0.052339252,
 0.00957487,
 0.058659572,
 0.011256735,
 0.035368454,
 0.058023218,
 -0.037726503,
 0.036206063,
 0.0362512,
 -0.020973828,
 -0.011257957,
 -0.019336306,
 -0.0072001894,
 0.057632796,
 0.004204795,
 -0.009985467,
 0.06119008,
 0.0016885438,
 0.0752836,
 0.0015567584,
 0.063829765,
 -0.0019720325,
 0.029755058,
 0.044302143,
 -0.00043386512,
 -0.026336223,
 0.009795321,
 -0.021034429,
 0.022844926,
 0.030783476,
 0.05415916,
 0.0479658,
 0.1345793,
 -0.060276333,
 0.061412193,
 -0.0005771393,
 0.004714105,
 0.021035615,
 0.012503666,
 0.007798911,
 0.03426409,
 0.06177317,
 0.006232359,
 0.025324414,
 0.06734841,
 -0.011082657,
 -0.018992916,
 -0.06712474,
 0.019105002,
 -0.054980963,
 -0.0413058,
 0.0348093,
 -0.01455394,
 0.017018301,
 0.037192434,
 0.067743905,
 -0.00065087073,
 0.02056822,
 -0.073584996,
 0.00551

In [46]:
await corpus_instance.n_paragraph_umap(5)

[10.299925, 4.379521, 14.433724]

In [ ]:
# Call the helper function to create your Corpus instance
# Make sure your Docker containers (db and api) are running!
w = await create_corpus_instance('spanish_1')

if w:
    print(f"\nCorpus instance 'w' created successfully for version: {w.version}")
    print(f"Author: {w.author}")
    print(f"Number of words: {w.n_words}")
    # You can now access other attributes or call methods on 'w'
    # For example:
    # word_frequencies = await w.word_freq(session) # You'd need to pass the session again for methods
    # print(f"Word frequencies (first 5): {list(word_frequencies.items())[:5]}")

In [ ]:
w = await Corpus.create('spanish_1')

In [15]:
from sqlalchemy.ext.asyncio import AsyncSession

async def open_request_(session: AsyncSession, sql_question: str):
    print('new_function')
    async with session.begin():
        result = await session.execute(text(sql_question))
        return result.scalar()

In [16]:
await open_request_("select count(*) from paragraph") 

TypeError: open_request_() missing 1 required positional argument: 'sql_question'

Error in open_request: AsyncSession.begin() missing 1 required positional argument: 'self'


TypeError: AsyncSession.begin() missing 1 required positional argument: 'self'

TypeError: open_request() missing 1 required positional argument: 'sql_question'

In [11]:
#s = await Corpus.create("german_1")

In [11]:
a = epub.read_epub("dissection_table/database/sources/Pedro Pâramo (Rulfo Juan)_german_(Z-Library).epub")

In [12]:
source_items = list(a.get_items_of_type(ebooklib.ITEM_DOCUMENT))

In [30]:
raw = BeautifulSoup(source_items[2].get_body_content(), 'html.parser')

In [ ]:
raw

In [32]:
paragraphs = [para.get_text() for para in raw.find_all('p')]

In [60]:
clear = paragraphs[44:-48]

In [61]:
clear[-1]

'Er stützte sich auf die Arme von Damiana Cisneros und versuchte zu gehen. Nach ein paar Schritten stürzte er, innerlich flehend, sagte aber kein einziges Wort. Er schlug hart auf die Erde auf und brach auseinander wie ein Haufen Steine.'

In [ ]:
meta_one = paragraphs[:44]

meta_two =  paragraphs[-48:-21]

meta_three = paragraphs[-21:]

In [ ]:
paragraphs

In [56]:
init_clear[-48:-46]

['Pedro Páramo –', 'dreißig Jahre danach']

In [2]:
import subprocess
import os
import sys
import shutil # For removing temporary files

def convert_doc_to_epub(input_doc_path: str, output_epub_path: str):
    """
    Converts a .doc file to an .epub file by first converting it to .docx
    using LibreOffice, and then using Pandoc for the final conversion.

    Args:
        input_doc_path (str): The full path to the input .doc file.
        output_epub_path (str): The full path for the output .epub file.
    """
    if not os.path.exists(input_doc_path):
        print(f"Error: Input file not found at '{input_doc_path}'")
        return

    # 1. Define temporary .docx path
    # Ensure the temporary file is in the same directory as the input, or a specified temp dir
    temp_docx_path = os.path.join(os.path.dirname(input_doc_path),
                                  os.path.basename(input_doc_path).replace(".doc", ".docx"))
    if input_doc_path == temp_docx_path: # Fallback if .doc isn't in name
        temp_docx_path = input_doc_path + ".docx"


    print(f"Attempting to convert '{input_doc_path}' to temporary DOCX: '{temp_docx_path}' using LibreOffice...")

    # Construct the LibreOffice conversion command
    libreoffice_command = [
        "libreoffice",
        "--headless",          # Run without a graphical interface
        "--convert-to", "docx", # Convert to DOCX format
        "--outdir", os.path.dirname(temp_docx_path) or ".", # Output to the same directory or current
        input_doc_path
    ]

    try:
        libreoffice_result = subprocess.run(libreoffice_command, capture_output=True, text=True, check=True)
        print("LibreOffice conversion to DOCX successful!")
        if libreoffice_result.stdout:
            print("LibreOffice stdout:")
            print(libreoffice_result.stdout)
        if libreoffice_result.stderr:
            print("LibreOffice stderr (warnings/info):")
            print(libreoffice_result.stderr)

        if not os.path.exists(temp_docx_path):
            print(f"Error: LibreOffice reported success but '{temp_docx_path}' was not created.")
            return

    except subprocess.CalledProcessError as e:
        print(f"Error during LibreOffice conversion: exited with code {e.returncode}")
        print("LibreOffice stdout:")
        print(e.stdout)
        print("LibreOffice stderr:")
        print(e.stderr)
        return
    except FileNotFoundError:
        print("Error: 'libreoffice' command not found. Please ensure LibreOffice is installed and in your system's PATH.")
        print("On Debian/Ubuntu: sudo apt-get install libreoffice-writer")
        print("On macOS (with Homebrew): brew install --cask libreoffice")
        print("On Windows: Download from libreoffice.org and ensure it's in PATH.")
        return
    except Exception as e:
        print(f"An unexpected error occurred during LibreOffice conversion: {e}")
        return

    # 2. Now use Pandoc to convert the temporary .docx to .epub
    print(f"\nAttempting to convert temporary DOCX '{temp_docx_path}' to EPUB: '{output_epub_path}' using Pandoc...")
    pandoc_command = [
        "pandoc",
        temp_docx_path, # Use the converted DOCX file
        "-o", output_epub_path,
        "--extract-media=.", # Extracts images and other media into a separate directory
        "--standalone"       # Produce a standalone document
    ]

    try:
        pandoc_result = subprocess.run(pandoc_command, capture_output=True, text=True, check=True)
        print("Pandoc conversion to EPUB successful!")
        print("Pandoc stdout:")
        print(pandoc_result.stdout)
        if pandoc_result.stderr:
            print("Pandoc stderr (warnings/info):")
            print(pandoc_result.stderr)
    except subprocess.CalledProcessError as e:
        print(f"Error during Pandoc conversion: exited with code {e.returncode}")
        print("Pandoc stdout:")
        print(e.stdout)
        print("Pandoc stderr:")
        print(e.stderr)
    except FileNotFoundError:
        print("Error: 'pandoc' command not found. Please ensure Pandoc is installed and in your system's PATH.")
        print("On Debian/Ubuntu: sudo apt-get install pandoc")
        print("On macOS (with Homebrew): brew install pandoc")
        print("On Windows: Download from pandoc.org and ensure it's in PATH.")
        return
    except Exception as e:
        print(f"An unexpected error occurred during Pandoc conversion: {e}")
    finally:
        # Clean up the temporary .docx file
        if os.path.exists(temp_docx_path):
            print(f"Cleaning up temporary file: '{temp_docx_path}'")
            os.remove(temp_docx_path)


In [6]:
ini =r"""dissection_table/database/sources/Pedro Pâramo (Rulfo Juan)_german_(Z-Library).doc""" 

In [7]:
out = "popo.epub"

In [8]:
convert_doc_to_epub(ini,out)


Attempting to convert 'dissection_table/database/sources/Pedro Pâramo (Rulfo Juan)_german_(Z-Library).doc' to temporary DOCX: 'dissection_table/database/sources/Pedro Pâramo (Rulfo Juan)_german_(Z-Library).docx' using LibreOffice...
LibreOffice conversion to DOCX successful!
LibreOffice stdout:
convert /home/jon/workshop/literary_stuff/pedro_paramo/pedro_paramo/dissection_table/database/sources/Pedro Pâramo (Rulfo Juan)_german_(Z-Library).doc as a Writer document -> /home/jon/workshop/literary_stuff/pedro_paramo/pedro_paramo/dissection_table/database/sources/Pedro Pâramo (Rulfo Juan)_german_(Z-Library).docx using filter : Office Open XML Text

LibreOffice stderr (warnings/info):


Attempting to convert temporary DOCX 'dissection_table/database/sources/Pedro Pâramo (Rulfo Juan)_german_(Z-Library).docx' to EPUB: 'popo.epub' using Pandoc...
Pandoc conversion to EPUB successful!
Pandoc stdout:

Pandoc stderr (warnings/info):
[WARNING] This document format requires a nonempty <title> elemen

In [2]:
# await open_request("DROP TABLE version CASCADE;")
# await open_request("DROP TABLE paragraph CASCADE;")

In [3]:
# await open_request("""SELECT text FROM paragraph WHERE n_paragraph = '2'
#                     AND version_name = 'spanish_1';
#         """)

In [4]:
await get_n_paragraph('spanish_1', 100000000)

"this pargraph: 100000000 doesn't exist"

In [ ]:
from dissection_table.database.ask_db import *
async def get_paragraph(n_paragraph):
    paragraph = get_n_paragraph(version, n_paragraph)
    return paragraph

In [12]:
await open_request("""
                select * from paragraph where paragraph.version_name = 'spanish_1'
                and paragraph.n_paragraph = 1
                """, fetch_as_dict=True)

[{'id': 1449,
  'version_name': 'spanish_1',
  'n_paragraph': 1,
  'text': 'Todavía antes me había dicho:',
  'n_words': 5}]

In [3]:
data_version = version("spanish_")
text = data_version["raw_text"]

VERSION :: spanish_ :: not found, returning original: "spanish_1"


In [4]:
text[:50]

'\nVine a Comala porque me dijeron que acá vivía mi '

In [5]:
start = time.time()

In [6]:
await feed_database()

Successfully performed bulk insert for 1447 items in Paragraph.
Successfully performed bulk insert for 1444 items in Paragraph.
Successfully performed bulk insert for 1447 items in Paragraph.
Successfully performed bulk insert for 1464 items in Paragraph.
Successfully performed bulk insert for 1447 items in Paragraph.
Successfully performed bulk insert for 1444 items in Paragraph.
Successfully performed bulk insert for 1388 items in Paragraph.


In [7]:
end = time.time()

In [8]:
print(end-start)

1.5437469482421875


In [12]:
await open_request("select * from version;")

[(1, 'spanish_1', 'Juan Rulfo', 1955, 'www.elejandria.com', None, 'NO_METADATA', '\nVine a Comala porque me dijeron que acá vivía mi padre, un tal Pedro Páramo. Mi madre me lo dijo. Y yo le prometí que vendría a verlo en cuanto ell ... (184088 characters truncated) ...  suplicando por dentro; pero sin decir una sola palabra. Dio un golpe seco contra la tierra y se fue desmoronando como si fuera un montón de piedras.')]

In [ ]:
await open_request("select * from paragraph;")

In [20]:
await open_request("DROP TABLE version CASCADE;")

In [ ]:
await open_request("select * from version;")

In [ ]:
game_interface.create_all(games_list)

In [2]:
s = version(source = "portuguese_1")

In [73]:
def _get_metadata(version:str = None):
    source = sources[version]
    source_items = list(source.get_items_of_type(ebooklib.ITEM_DOCUMENT))
    metadata = []

    for ind,item in enumerate(source_items):
        data_text = BeautifulSoup(item.get_body_content(), 'html.parser')
        paragraphs = [para.get_text() for para in data_text.find_all('p')]
        paragraphs = [x.replace('\xa0', '') for x in paragraphs]
        if len(paragraphs) == 0:
            continue
        if len(paragraphs) > 100:
            raw += paragraphs
            continue
        metadata += [f"########## ITEM {ind}"]
        metadata += paragraphs
    return metadata
        
portugues_1_hard_coded_data = {}
portugues_1_hard_coded_data["metadata"] = _get_metadata(version = "portuguese_1")
portugues_1_hard_coded_data["author"] = "Eric Nepomuceno"
portugues_1_hard_coded_data["year"] = 2009
portugues_1_hard_coded_data["editorial"] = "Edições BestBolso"
portugues_1_hard_coded_data["ISBN"] = 9788577991167

spanish_1_hard_coded_data = {}
spanish_1_hard_coded_data["metadata"] = "NO_METADATA"
spanish_1_hard_coded_data["author"] = "Juan Rulfo"
spanish_1_hard_coded_data["year"] = 1955
spanish_1_hard_coded_data["editorial"] = "www.elejandria.com"
spanish_1_hard_coded_data["ISBN"] = None

spanish_2_hard_coded_data = {}
spanish_2_hard_coded_data["metadata"] = _get_metadata(version = "spanish_2")
spanish_2_hard_coded_data["author"] = "Juan Rulfo"
spanish_2_hard_coded_data["year"] = 2011
spanish_2_hard_coded_data["editorial"] = "feather"
spanish_2_hard_coded_data["ISBN"] = None

english_1_hard_coded_data = {}
english_1_hard_coded_data["metadata"] = _get_metadata(version = "english_1")
english_1_hard_coded_data["author"] = "Margaret Sayers Peden"
english_1_hard_coded_data["year"] = 1994
english_1_hard_coded_data["editorial"] = "Grove Press"
english_1_hard_coded_data["ISBN"] = 9780802133908

english_2_hard_coded_data = {}
english_2_hard_coded_data["metadata"] = _get_metadata(version = "english_2")
english_2_hard_coded_data["author"] = "Douglas J. Weatherford"
english_2_hard_coded_data["year"] = 2023
english_2_hard_coded_data["editorial"] = "Grove Press"
english_2_hard_coded_data["ISBN"] = 9780802160935

italian_1_hard_coded_data = {}
italian_1_hard_coded_data["metadata"] = _get_metadata(version = "italian_1")
italian_1_hard_coded_data["author"] = "Paolo Collo"
italian_1_hard_coded_data["year"] = 2014
italian_1_hard_coded_data["editorial"] = "Giulio Einaudi editore s.p.a."
italian_1_hard_coded_data["ISBN"] = 9788858440247

french_1_hard_coded_data = {}
french_1_hard_coded_data["metadata"] = _get_metadata(version = "french_1")
french_1_hard_coded_data["author"] = "Gabriel Iaculli"
french_1_hard_coded_data["year"] = 2005
french_1_hard_coded_data["editorial"] = "Éditions Gallimard, 2005,"
french_1_hard_coded_data["ISBN"] = 9782070379538


In [74]:
french_1_hard_coded_data["metadata"][0]

['########## ITEM 2',
 'JUAN RULFO',
 '',
 'PEDRO PÁRAMO',
 '',
 'roman',
 '',
 '',
 '',
 'Nouvelle traduction de l’espagnol (Mexique)',
 'par Gabriel Iaculli',
 '',
 '',
 '',
 'nrf',
 'GALLIMARD',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Titre original:',
 'PEDRO PÁRAMO',
 '© Heirs of Juan Rulfo, 1955.',
 '© Éditions Gallimard, 2005, pour la traduction française.']

In [34]:
source = sources["spanish_1"]
source_items = list(source.get_items_of_type(ebooklib.ITEM_DOCUMENT))

In [35]:
data_text = BeautifulSoup(source_items[0].get_body_content(), 'html.parser')

In [36]:
paragraphs = [para.get_text() for para in data_text.find_all('p')]
paragraphs = [x.replace('\xa0', '') for x in paragraphs]

In [41]:
paragraphs[-5:]

['—Voy para allá. Ya voy.',
 'Se apoyó en los brazos de Damiana Cisneros e hizo intento de caminar. Después de unos cuantos pasos cayó, suplicando por dentro; pero sin decir una sola palabra. Dio un golpe seco contra la tierra y se fue desmoronando como si fuera un montón de piedras.',
 '',
 '',
 '']

In [28]:
len(paragraphs)

0

In [33]:
source_items

In [ ]:
_get_metadata("portuguese_1")

In [12]:
def get_metadata(version:str = None):
    source = sources[version]
    metadata = []
    for ind, turn in enumerate([3,4,5,7]):
        data_text = list(source.get_items_of_type(ebooklib.ITEM_DOCUMENT))[turn]
        data_text = BeautifulSoup(data_text.get_body_content(), 'html.parser')
        paragraphs = [para.get_text() for para in data_text.find_all('p')]
        paragraphs = [x.replace('\xa0', '') for x in paragraphs]
        metadata += [f"########## ITEM {ind}"]
        metadata += paragraphs
    return "\n".join([x for x in metadata])

portugues_hard_coded_data = {}
portugues_hard_coded_data["author"] = "Eric Nepomuceno"
portugues_hard_coded_data["year"] = 2009
portugues_hard_coded_data["editorial"] = "Edições BestBolso"
portugues_hard_coded_data["ISBN"] = 9788577991167
portugues_hard_coded_data["metadata"] = get_metadata(version = "portuguese_1")


In [ ]:
print(portugues_hard_coded_data["metadata"])

In [63]:
source = sources["portuguese_1"]

In [64]:
whole_text = list(source.get_items_of_type(ebooklib.ITEM_DOCUMENT))[7]
whole_text = BeautifulSoup(whole_text.get_body_content(), 'html.parser')

In [65]:
paragraphs = [para.get_text() for para in whole_text.find_all('p')]
paragraphs = [x.replace('\xa0', '') for x in paragraphs]

In [66]:
essay_about = paragraphs #3+4+5+7

In [67]:
paragraphs

['Você pode adquirir os títulos da BestBolso através do Marketing Direto do Grupo Editorial Record.',
 '• Telefone: (21) 2585-2002',
 '(de segunda a sexta-feira, das 8h30 às 18h)',
 '• E-mail: mdireto@record.com.br',
 '• Fax: (21) 2585-2010',
 'Entre em contato conosco caso tenha alguma dúvida, precise de informações ou queira se cadastrar para receber nossos informativos de lançamentos e promoções.',
 'Nossos sites:',
 'www.edicoesbestbolso.com.br',
 'www.record.com.br']

In [47]:
metadata = '\n'.join([x for x in paragraphs]) #3

In [2]:
bad_version = version("pipi")

VERSION :: pipi :: not found, returning original: "spanish_1"


In [3]:
portuguese = version('portuguese_1')

In [4]:
portuguese[0]

'VIM A COMALA porque me disseram que aqui vivia meu pai, um tal de Pedro Páramo.'

In [5]:
spanish_1 = version("spanish_1")

In [6]:
spanish_1[0]

'Vine a Comala porque me dijeron que acá vivía mi padre, un tal Pedro Páramo. Mi madre me lo dijo. Y yo le prometí que vendría a verlo en cuanto ella muriera. Le apreté sus manos en señal de que lo haría; pues ella estaba por morirse y yo en plan de prometerlo todo. «No dejes de ir a visitarlo —me recomendó—. Se llama de otro modo y de este otro. Estoy segura de que le dará gusto conocerte». Entonces no pude hacer otra cosa sino decirle que así lo haría, y de tanto decírselo se lo seguí diciendo aun después que a mis manos les costó trabajo zafarse de sus manos muertas.'

In [7]:
spanish_2 = version("spanish_2")

In [8]:
spanish_2[0]

'Vine a Comala porque me dijeron que acá vivía mi padre, un tal Pedro Páramo. Mi madre me lo dijo. Y yo le prometí que vendría a verlo en cuanto ella muriera. Le apreté sus manos en señal de que lo haría; pues ella estaba por morirse y yo en plan de prometerlo todo. «No dejes de ir a visitarlo —me recomendó—. Se llama de otro modo y de este otro. Estoy segura de que le dará gusto conocerte.» Entonces no pude hacer otra cosa sino decirle que así lo haría, y de tanto decírselo se lo seguí diciendo aun después que a mis manos les costó trabajo zafarse de sus manos muertas.'

In [9]:
english_1 = version("english_1")

In [10]:
english_1[-1]

"He supported himself on Damiana Cisnero's arm and tried to walk. After a few steps he fell; inside, he was begging for help, but no words were audible. He fell to the ground with a thud, and lay there, collapsed like a pile of rocks."

In [11]:
english_2 = version("english_2")

In [12]:
english_2[-1]

'He supported himself on Damiana Cisneros’s arms and tried to walk. After a few steps he fell, pleading on the inside, but without saying a word. He hit the ground with a hollow thud, crumbling as if he were a pile of rocks.'

In [13]:
italian_1 = version("italian_1")

In [14]:
italian_1[-1]

'Si appoggiò alle braccia di Damiana Cisneros e provò a camminare. Dopo pochi passi cadde, supplicando dentro di sé; ma senza dire una sola parola. Diede un colpo secco contro la terra e si sgretolò come se fosse un mucchio di pietre.'

In [15]:
french_1 = version("french_1")

In [16]:
french_1[0]

'Je suis venu à Comala parce que j’ai appris que mon père, un certain Pedro Páramo, y vivait. C’est ma mère qui me l’a dit. Et je lui ai promis d’aller le voir quand elle serait morte. J’ai pressé ses mains pour lui assurer que je le ferais; elle se mourait et j’étais prêt à lui promettre n’importe quoi. «Ne manque pas d’aller le trouver, m’a-t-elle recommandé. Il porte tel prénom et tel nom. Je suis sûre qu’il sera content de te connaître.» Dans ces conditions, il a bien fallu lui dire que je n’y manquerais pas, et, à force de le lui répéter, j’y étais encore après avoir, non sans peine, détaché mes mains de ses mains mortes.'

In [22]:
"\xa0" in french_1[-1]

False

In [72]:
source = sources["french_1"]

In [73]:
general = list(source.get_items_of_type(ebooklib.ITEM_DOCUMENT))

In [74]:
a = BeautifulSoup(general[3].get_body_content(), 'html.parser')
b = BeautifulSoup(general[4].get_body_content(), 'html.parser')

In [75]:
b = [para.get_text() for para in b.find_all('p')]

In [79]:
b[-1]

'Il s’est appuyé sur le bras de Damiana Cisneros et a essayé de marcher. Après avoir fait quelques pas, il est tombé, une supplication tout au fond de lui-même, mais sans dire un mot. Il a heurté la terre d’un coup sec et s’est écroulé comme un tas de pierres.'

In [77]:
a = [para.get_text() for para in a.find_all('p')]

In [80]:
complete = a[4:] + b

In [82]:
complete[-1]

'Il s’est appuyé sur le bras de Damiana Cisneros et a essayé de marcher. Après avoir fait quelques pas, il est tombé, une supplication tout au fond de lui-même, mais sans dire un mot. Il a heurté la terre d’un coup sec et s’est écroulé comme un tas de pierres.'

In [ ]:
source = sources["spanish_2"]
        whole_text = list(source.get_items_of_type(ebooklib.ITEM_DOCUMENT))[0]
        whole_text = BeautifulSoup(whole_text.get_body_content(), 'html.parser')
        paragraphs = [para.get_text() for para in whole_text.find_all('p')][:-3]
        return paragraphs[1:]

In [6]:
whole_text = list(source.get_items_of_type(ebooklib.ITEM_DOCUMENT))        

In [7]:
text = BeautifulSoup(whole_text[0].get_body_content(), 'html.parser')

In [9]:
paragraphs = [para.get_text() for para in text.find_all('p')]

In [17]:
paragraphs[:-3]

['—Voy para allá. Ya voy.',
 'Se apoyó en los brazos de Damiana Cisneros e hizo intento de caminar. Después de unos cuantos pasos cayó, suplicando por dentro; pero sin decir una sola palabra. Dio un golpe seco contra la tierra y se fue desmoronando como si fuera un montón de piedras.']

In [ ]:
whole_text = BeautifulSoup(whole_text.get_body_content(), 'html.parser')
        paragraphs = [para.get_text() for para in whole_text.find_all('p')][:-2]

In [3]:
sources

{'portuguese_1': <ebooklib.epub.EpubBook at 0x75d97777a420>,
 'spanish_1': <ebooklib.epub.EpubBook at 0x75d9653ffe00>,
 'spanish_2': <ebooklib.epub.EpubBook at 0x75d9653721b0>,
 'english_1': <ebooklib.epub.EpubBook at 0x75d964e3deb0>,
 'english_2': <ebooklib.epub.EpubBook at 0x75d964e3e810>,
 'italian_1': <ebooklib.epub.EpubBook at 0x75d964e3e8d0>,
 'french_1': <ebooklib.epub.EpubBook at 0x75d964e3fe90>}

In [7]:
a = list(sources["spanish_1"].get_items_of_type(ebooklib.ITEM_DOCUMENT))

In [ ]:
BeautifulSoup(a[0].get_body_content(), 'html.parser')

In [12]:
spanish_1[-3]

'Apoiou-se nos braços de Damiana Cisneros e fez a tentativa de caminhar. Depois de alguns tantos passos caiu, suplicando por dentro; mas sem dizer uma única palavra. Deu uma batida seca contra a terra e foi se desmoronando como se fosse um montão de pedras.'

In [ ]:
def get_raw_portugues(potuguese_source: dict= {}):
    whole_text = list(potuguese_source.get_items_of_type(ebooklib.ITEM_DOCUMENT))[6]
    whole_text = BeautifulSoup(whole_text.get_body_content(), 'html.parser')
    paragraphs = [para.get_text() for para in whole_text.find_all('p')]
    return paragraphs

In [2]:
potuguese_source = epub.read_epub("dissection_table/database/sources/Pedro Páramo (Juan Rulfo [Rulfo, Juan])_portugues_(Z-Library).epub")
spanish_source_1 = epub.read_epub("dissection_table/database/sources/Pedro Paramo (Juan Rulfo)_espanol_(Z-Library).epub")
spanish_source_2 = epub.read_epub("dissection_table/database/sources/Pedro Páramo (Juan Rulfo)_first_espanol_(Z-Library).epub")
english_source_1 = epub.read_epub("dissection_table/database/sources/Pedro Paramo (Juan Rulfo)_peden_(Z-Library).epub")
english_source_2 = epub.read_epub("dissection_table/database/sources/Pedro Paramo (Juan Rulfo)_weatherford_ (Z-Library).epub")
italian_source_1 = epub.read_epub("dissection_table/database/sources/Pedro Páramo (Juan Rulfo)_italian_(Z-Library).epub")
frech_source_1   = epub.read_epub("dissection_table/database/sources/Pedro Páramo (Juan Rulfo)_french_(Z-Library).epub")

/home/jon/anaconda3/lib/python3.12/site-packages/ebooklib/epub.py:1347: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [14]:
# items = list(potuguese_source.get_items_of_type(ebooklib.ITEM_DOCUMENT))

# o_items = BeautifulSoup(items[6].get_body_content(), 'html.parser')

# p_p = [para.get_text() for para in o_items.find_all('p')]

In [3]:
from bs4 import BeautifulSoup
def chapter_to_str(chapter):
    soup = BeautifulSoup(chapter.get_body_content(), 'html.parser')
    text = [para.get_text() for para in soup.find_all('p')]
    return text

In [6]:
def get_raw_portugues(potuguese_source: dict= {}):
    whole_text = list(potuguese_source.get_items_of_type(ebooklib.ITEM_DOCUMENT))[6]
    whole_text = BeautifulSoup(whole_text.get_body_content(), 'html.parser')
    paragraphs = [para.get_text() for para in whole_text.find_all('p')]
    return paragraphs

In [7]:
whole_text = get_raw_portugues(potuguese_source)

In [ ]:
whole_text

In [17]:
get_raw_portugues(potuguese_source)

TypeError: 'NoneType' object is not callable

In [7]:
get_raw_spanish()

NameError: name 'texts' is not defined

In [51]:
spanish_source_1[0]

In [12]:
items = list(book.get_items_of_type(ebooklib.ITEM_DOCUMENT))

In [13]:
items

In [26]:
type(texts)

dict

In [40]:

def get_raw_spanish(spanish_text: dict = {}):
    pass

In [29]:
portugues = get_raw_portugues(texts)

In [32]:
a = portugues.split('\n')

In [35]:
len(a[0])

188208

In [ ]:
a[0]

In [ ]:
texts['Text/007.html']